In [1]:
import numpy as np
import pandas as pd
import math
import random
import matplotlib.pyplot as plt

In [17]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [69]:
# Optimal bids from the 5 implemented strategies

LR_linear = pd.read_csv('LinearBids.csv', index_col=0 )
XGB_linear = pd.read_csv('optimalbids_validation_linear.csv', index_col=0)
XGB_square = pd.read_csv('optimalbids_validation_square.csv', index_col=0)
XGB_ortb1 = pd.read_csv('optimalbids_validation_ortb1.csv', index_col=0)
XGB_ortb2 = pd.read_csv('optimalbids_validation_ortb2.csv', index_col=0)

In [70]:
LR_linear.shape, XGB_linear.shape, XGB_square.shape, XGB_ortb1.shape, XGB_ortb2.shape

((303925, 1), (303925, 1), (303925, 1), (303925, 1), (303925, 1))

In [71]:
# Import validation set

validation = pd.read_csv('validation.csv')

In [18]:
validation.head()

,click,weekday,hour,bidid,userid,useragent,IP,region,city,adexchange,domain,url,urlid,slotid,slotwidth,slotheight,slotvisibility,slotformat,slotprice,creative,bidprice,payprice,keypage,advertiser,usertag
0,0,4,20,bbcb813b6166538503d8b33a5602d7d72f6019dc,663169f66491c98c69f1f94a5c48fa34aa9fe06f,windows_ie,211.144.203.*,79,79,1.0,trqRTvFRLpscFU,5ede68cc7542a34777fb8a1574856b6f,NaN,mm_10027070_2459574_9659312,160,600,2,1,0,cb7c76e7784031272e37af8e7e9b062c,300,23,bebefa5efe83beee17a3d245e7c5085b,1458,"13866,10111"
1,0,1,21,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,a23f0cfab6592c137f796e68fa752fceba08bb1b,windows_chrome,58.247.250.*,79,79,1.0,trqRTu1YP5scFsf,cff326604db401e7d4f895fd747f7e91,NaN,mm_10075660_3500949_11453278,950,90,0,1,0,c938195f9e404b4f38c7e71bf50263e5,238,75,a3becb967c015e9acb5e72f22b100ef8,3476,"10075,10057,10024,10052,10083,10063,10006,10110"
2,0,4,8,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,5105a013ea54a9706146033e6a138d5234a3803e,windows_ie,117.12.111.*,2,2,2.0,trqRTv14MqTR1m58uG,93f242f025dc7d424c4ba7002b177df7,NaN,2996225656,300,250,2,0,5,911b2d84826786018761e8c0b0a3a60c,238,65,d29e59bf0f7f8243858b8183f14d4412,3358,"13866,10024,10059,10063,13776,10083,10006,10111"
3,0,5,15,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,577e9a768ce17e73c61ba7022db927f9ee761830,windows_chrome,61.187.224.*,201,205,2.0,trqRTvdJMoFoFoc4JKn,40c388f45dbc0e1fc53a8c6c81305de5,NaN,3481684581,336,280,2,0,5,dc0998c10f8f0b623b5d949e8272e4c7,238,6,d29e59bf0f7f8243858b8183f14d4412,3358,"13866,10057,10006,10063,10110"
4,0,1,18,0899bf144249458ea9c89188473694bf44c7ca15,e4bf79c562745d671b19f2edf7fda89c2e25987f,windows_chrome,117.41.145.*,134,135,2.0,tKFJeun0BouoXOl4JKTI,fcf53e0e5f09aa8d7e1c9d986cea321,NaN,1120607322,200,200,1,0,5,23485fcd23122d755d38f8c89d46ca56,249,5,7e0e909ea89f707bc20f9585c9072081,3476,"10006,10063,13776,10110"


In [208]:
validation_part = validation[['click', 'bidid', 'slotprice']]

In [209]:
# Create two new columns, in which we will store the winner of the impression and click

validation_part['winner_imp'] = 0
validation_part['winner_clk'] = 0


C:\Users\apogn\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\apogn\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [210]:
validation_part.shape

(303925, 5)

In [211]:
# Concatenate the datasets

validation_part.loc[:, 'LR'] = LR_linear.values
validation_part.loc[:, 'XGB_lnr'] = XGB_linear.values
validation_part.loc[:, 'XGB_sqr'] = XGB_square.values
validation_part.loc[:, 'XGB_ortb1'] = XGB_ortb1.values
validation_part.loc[:, 'XGB_ortb2'] = XGB_ortb2.values


In [212]:
validation_part.loc[:, 'LR_bud'] = 6250000
validation_part.loc[:, 'XGB_lnr_bud'] = 6250000
validation_part.loc[:, 'XGB_sqr_bud'] = 6250000
validation_part.loc[:, 'XGB_ortb1_bud'] = 6250000
validation_part.loc[:, 'XGB_ortb2_bud'] = 6250000

In [213]:
# result = validation_part[['LR','XGB_lnr', 'XGB_sqr', 'XGB_ortb1', 'XGB_ortb2']]

# validation_part['winner_imp'] = result.idxmax(axis=1)

In [224]:
validation_part.head()

,click,bidid,slotprice,winner_imp,winner_clk,LR,XGB_lnr,XGB_sqr,XGB_ortb1,XGB_ortb2,LR_bud,XGB_lnr_bud,XGB_sqr_bud,XGB_ortb1_bud,XGB_ortb2_bud
0,0,bbcb813b6166538503d8b33a5602d7d72f6019dc,0,0,0,50.133784,28.304638,11.512002,46.189816,52.254654,6250000,6250000,6250000,6250000,6250000
1,0,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,0,0,0,20.858784,18.620140,4.981975,37.462376,45.445360,6250000,6250000,6250000,6250000,6250000
2,0,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,5,0,0,28.953844,46.608143,31.214706,59.273580,61.707176,6250000,6250000,6250000,6250000,6250000
3,0,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,5,0,0,26.981120,27.313885,10.720193,45.374107,51.637610,6250000,6250000,6250000,6250000,6250000
4,0,0899bf144249458ea9c89188473694bf44c7ca15,5,0,0,31.311401,20.207418,5.867557,39.026730,46.701890,6250000,6250000,6250000,6250000,6250000


In [225]:
test = validation_part.head(100)

In [226]:
test.shape

(100, 15)

In [227]:
# "%%pixie_debugger

mmm = ['slotprice','LR', 'XGB_lnr', 'XGB_sqr', 'XGB_ortb1', 'XGB_ortb2']
endBudget = pd.Series(np.zeros(5), index = mmm[1:])
for i, row in test.iterrows():
    
    LR = row ['LR']
    XGBlinear = row ['XGB_lnr']
    XGBsquare = row ['XGB_sqr']
    XGBortb1 = row ['XGB_ortb1']
    XGBortb2 = row ['XGB_ortb2']
    sltprc = row ['slotprice']
    
    
    val = [sltprc, LR, XGBlinear, XGBsquare, XGBortb1, XGBortb2]
    result = pd.Series(val, index = mmm)
    
#     result.loc[0] = [sltprc, LR, XGBlinear, XGBsquare, XGBortb1, XGBortb2]
#     result = [sltprc, LR, XGBlinear, XGBsquare, XGBortb1, XGBortb2]
    
#     Checking if we exceeded the remaining budget for each agent
    if(LR > test.iloc[i,10]): #LR agent
        result = result.drop('LR')
    
    if(XGBlinear > test.iloc[i,11]): #XGB_lnr agent
        result = result.drop('XGB_lnr')
    
    if(XGBsquare > test.iloc[i,12]): #XGB sqr agent
        result = result.drop('XGB_sqr')
    
    if(XGBortb1 > test.iloc[i,13]): #XGB ortb1 agent
        result = result.drop('XGB_ortb1')
    
    if(XGBortb2 > test.iloc[i,14]): #XGB ortb2 agent
        result = result.drop('XGB_ortb2')  
    
    agent_max_bid = result.idxmax()
    test['winner_imp'] = agent_max_bid   # winner of the impression
    
    
    if (test.iloc[i,0] == 1):
        test['winner_clk'] = agent_max_bid
    else: 
        test['winner_clk'] = 'None'

    
#     Reduction of the budget for the winner of the impression
    if i != test.shape[0]-1:
        test.iloc[i+1,10] = test.iloc[i,10]
        test.iloc[i+1,11] = test.iloc[i,11]
        test.iloc[i+1,12] = test.iloc[i,12] 
        test.iloc[i+1,13] = test.iloc[i,13] 
        test.iloc[i+1,14] = test.iloc[i,14] 

        if(agent_max_bid == 'LR'):
            test.iloc[i+1,10] = test.iloc[i,10] - LR

        if(agent_max_bid == 'XGB_lnr'):
            test.iloc[i+1,11] = test.iloc[i,11] - XGBlinear

        if(agent_max_bid == 'XGB_sqr'):
            test.iloc[i+1,12] = test.iloc[i,12] - XGBsquare

        if(agent_max_bid == 'XGB_ortb1'):
            test.iloc[i+1,13] = test.iloc[i,13] - XGBortb1

        if(agent_max_bid == 'XGB_ortb2'):
            test.iloc[i+1,14] = test.iloc[i,14] - XGBortb2
    else: 
        endBudget[0] = test.iloc[i,10]
        endBudget[1] = test.iloc[i,11]
        endBudget[2] = test.iloc[i,12] 
        endBudget[3] = test.iloc[i,13] 
        endBudget[4] = test.iloc[i,14] 

        if(agent_max_bid == 'LR'):
            endBudget[0] = test.iloc[i,10] - LR

        if(agent_max_bid == 'XGB_lnr'):
            endBudget[1] = test.iloc[i,11] - XGBlinear

        if(agent_max_bid == 'XGB_sqr'):
            endBudget[2] = test.iloc[i,12] - XGBsquare

        if(agent_max_bid == 'XGB_ortb1'):
            endBudget[3] = test.iloc[i,13] - XGBortb1

        if(agent_max_bid == 'XGB_ortb2'):
            endBudget[4] = test.iloc[i,14] - XGBortb2
       
        
        

C:\Users\apogn\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\apogn\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\apogn\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [228]:
endBudget

LR           6.247319e+06
XGB_lnr      6.250000e+06
XGB_sqr      6.238577e+06
XGB_ortb1    6.249858e+06
XGB_ortb2    6.247232e+06
dtype: float64

In [235]:
# stat = validation_part[['click']].groupby(['winner_clk']).sum()#
# stat = pd.DataFrame()
# stat['impressions'] = validation_part.groupby(['winner_imp']).size()


# stat.head()

test

,click,bidid,slotprice,winner_imp,winner_clk,LR,XGB_lnr,XGB_sqr,XGB_ortb1,XGB_ortb2,LR_bud,XGB_lnr_bud,XGB_sqr_bud,XGB_ortb1_bud,XGB_ortb2_bud
0,0,bbcb813b6166538503d8b33a5602d7d72f6019dc,0,XGB_ortb2,None,50.133784,28.304638,11.512002,46.189816,52.254654,6.250000e+06,6250000,6.250000e+06,6.250000e+06,6.250000e+06
1,0,5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d,0,XGB_ortb2,None,20.858784,18.620140,4.981975,37.462376,45.445360,6.250000e+06,6250000,6.250000e+06,6.250000e+06,6.249948e+06
2,0,f6ece71dae81d6b16bfb24ad6dd5611472d4c673,5,XGB_ortb2,None,28.953844,46.608143,31.214706,59.273580,61.707176,6.250000e+06,6250000,6.250000e+06,6.250000e+06,6.249902e+06
3,0,b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb,5,XGB_ortb2,None,26.981120,27.313885,10.720193,45.374107,51.637610,6.250000e+06,6250000,6.250000e+06,6.250000e+06,6.249841e+06
4,0,0899bf144249458ea9c89188473694bf44c7ca15,5,XGB_ortb2,None,31.311401,20.207418,5.867557,39.026730,46.701890,6.250000e+06,6250000,6.250000e+06,6.250000e+06,6.249789e+06
5,0,f4c6a9a3b1db6da035c7e2a36d4f5e389095beca,20,XGB_ortb2,None,152.472128,35.658840,18.271328,51.845060,56.437300,6.250000e+06,6250000,6.250000e+06,6.250000e+06,6.249742e+06
6,0,786a2940f225482dc04d455dc7a7fea436b02e03,0,XGB_ortb2,None,93.022817,81.904740,96.394840,78.577090,74.466150,6.249848e+06,6250000,6.250000e+06,6.250000e+06,6.249742e+06
7,0,17ada861c6ed0616f88312ba0d3d76c0f95b4940,0,XGB_ortb2,None,61.894452,107.167570,165.029830,89.882970,81.447845,6.249848e+06,6250000,6.249904e+06,6.250000e+06,6.249742e+06
8,0,6abd8ef3eb678b1afe67dc0cff0aa58413fbf29a,5,XGB_ortb2,None,32.906673,32.272490,14.965825,49.321625,54.590790,6.249848e+06,6250000,6.249739e+06,6.250000e+06,6.249742e+06
9,0,442ffca310e325abc255e8ff97d9812051336acd,50,XGB_ortb2,None,9.530869,24.647877,8.729606,43.102554,49.899452,6.249848e+06,6250000,6.249739e+06,6.250000e+06,6.249688e+06


In [234]:
test.winner_imp.value_counts()

XGB_ortb2    100
Name: winner_imp, dtype: int64